In [3]:
import os, glob, joblib, torch, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
import glob, pandas as pd, re, tqdm, collections, json, torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from tqdm import tqdm
import re, torch
from wordfreq import zipf_frequency
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, torch.nn.functional as F
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import re
import emoji
from ftfy import fix_text
from datasets import Dataset
from torch.utils.data import DataLoader

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import re, regex
import html, emoji, contractions
from ftfy import fix_text
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

In [4]:
# LEX_PATH = Path("roman_hindi_lexicon.txt")
# if LEX_PATH.exists() and LEX_PATH.stat().st_size > 100_000:
#     print("✅ large lexicon already present:", LEX_PATH.resolve())
# else:
#     print("🔧 building offline roman-Hindi lexicon from your dataset …")
#     # --- reload LID once ---
#     lid_name = "papluca/xlm-roberta-base-language-detection"
#     lid_tok  = AutoTokenizer.from_pretrained(lid_name)
#     lid_net  = AutoModelForSequenceClassification.from_pretrained(lid_name)

#     word_re  = re.compile(r"\b[a-z]{3,}\b", flags=re.I)
#     vowel_pat= re.compile(r"(aa|ee|ii|oo|uu|ai|au)", re.I)

#     def is_hinglish_candidate(tok):
#         return vowel_pat.search(tok) is not None

#     counter = collections.Counter()

#     csv_paths = glob.glob(r"labeled_batches\labeled_batch_*.csv")
#     for path in tqdm.tqdm(csv_paths, desc="scanning"):
#         df = pd.read_csv(path, usecols=["Transliterated_Comment"])
#         for text in df["Transliterated_Comment"].astype(str):
#             for tok in word_re.findall(text.lower()):
#                 if is_hinglish_candidate(tok):
#                     counter[tok] += 1

#     # keep top 15k candidates
#     candidates = [w for w,_ in counter.most_common(15_000)]

#     # batch-predict language
#     roman_hi = set()
#     B = 64
#     for i in tqdm.tqdm(range(0, len(candidates), B), desc="LID"):
#         chunk = candidates[i:i+B]
#         inputs= lid_tok(chunk, padding=True, truncation=True, return_tensors="pt")
#         with torch.inference_mode():
#             probs = F.softmax(lid_net(**inputs).logits, dim=-1)
#         idxs  = probs.argmax(dim=-1)
#         langs = [lid_net.config.id2label[j.item()] for j in idxs]
#         confs = probs.max(dim=-1).values
#         for w,l,c in zip(chunk, langs, confs):
#             if l in ("hi","ur") and c > 0.60:
#                 roman_hi.add(w)

#     # add your manual extra list
#     roman_hi.update("""
#         kya haan nahi kaise kyu kyun bhai kitna kyon chal sahi banda khud
#         sab theek acha accha pyaar dil zindagi
#     """.split())

#     # save
#     LEX_PATH.write_text("\n".join(sorted(roman_hi)))
#     print(f"✅ saved {len(roman_hi):,} roman-Hindi words → {LEX_PATH.resolve()}")

In [5]:
lid_name = "papluca/xlm-roberta-base-language-detection"
lid_tok  = AutoTokenizer.from_pretrained(lid_name)
lid_net  = AutoModelForSequenceClassification.from_pretrained(lid_name)

word_re = re.compile(r"\w+", flags=re.UNICODE)
DEV_START, DEV_END = 0x0900, 0x097F

roman_hi = set(open("roman_hindi_lexicon.txt").read().split())

def is_devanagari(tok: str) -> bool:
    return any(DEV_START <= ord(c) <= DEV_END for c in tok)

@torch.inference_mode()
def hf_lang(token):
    inputs = lid_tok(token, return_tensors="pt")
    logits = lid_net(**inputs).logits
    probs  = F.softmax(logits, dim=-1)
    idx    = probs.argmax().item()
    lang   = lid_net.config.id2label[idx]
    return lang, probs[0, idx].item()       # ('hi', 0.78)

def token_is_hindi(tok: str) -> bool:
    tok_lc = tok.lower()
    if is_devanagari(tok):                     # Stage 1
        return True
    if tok_lc in roman_hi:                     # Stage 2
        return True
    lang, p = hf_lang(tok_lc)            # Stage 3
    if lang in {"hi", "ur"} and p > 0.5:
        return True
    return zipf_frequency(tok_lc, "en") < 2.5 # Stage 4 (rarity)

def hindi_ratio(text: str) -> float:
    toks = word_re.findall(text)
    if not toks:
        return 0.0
    hi = sum(token_is_hindi(t) for t in toks)
    return hi / len(toks)


In [6]:
print(hindi_ratio("kya haal hai?"))                  # → ~1.0
print(hindi_ratio("good morning everyone"))          # → ~0.0
print(hindi_ratio("kya bol rahe ho boss good job"))   # → ~0.4-0.6 (mixed)


1.0
0.0
0.42857142857142855


In [7]:
ROOT = r"D:/University of Illinois Chicago/Classes/CS521/cs521"
BATCH_DIR = os.path.join(ROOT, "labeled_batches")
BASE_DIR  = os.path.join(ROOT, "baseline_models")
HING_DIR  = os.path.join(ROOT, "sarcasm_hingbert_model")
DEVANAGARI_TO_ROMAN = True
LATIN_BLOCK= r'\p{Latin}'
DEVANAGARI_BLOCK = r'\p{Devanagari}'

In [8]:
censored_swear_words = {
    # Handles s!@#, s#!t, s**t, etc., variations of "shit"
    r'\b[Ss][^A-Za-z\s]{3,4}\b': 'shit',
    r'[Ss][\!\@\#\$\%\^\&\*]*[i1\|][\!\@\#\$\%\^\&\*]*[t]+': 'shit',
    r'[Ss][#!@$%^&*+\-=\[\]{};:\'",.<>?/\\|_~`]{2,}[Tt]\b': 'shit',


    # Handles f*ck, f@ck, f**k, f!ck, F*#k, F***, etc., variations of "fuck"
    r'f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k]+': 'fuck',
    r'f[\!\@\#\$\%\^\&\*]*[\*]+[c][k]': 'fuck',                # Handles f*ck
    r'f[\!\@\#\$\%\^\&\*]*[\*]+[k]': 'fuck',                   # Handles f***
    r'f[\!\@\#\$\%\^\&\*]*[\*#\!]{1,}[c]?[kz]?': 'fuck',       # Handles F*#k, F***
    r'f[\!\@\#\$\%\^\&\*]*[\w@#$%^&*]+ing\b': 'fucking',        # Hnadles f#%^ing

    # Handles b*tch, b!tch, b1tch, etc., variations of "bitch"
    r'b[\!\@\#\$\%\^\&\*]*[i1\|][\!\@\#\$\%\^\&\*]*[t][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[h]+': 'bitch',

    # Handles "fuck" with various suffixes (e.g., "f***'z" -> "fucks")
    r'f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k]+[\'’]?[zs]?': 'fucks',

    # Handles muthaf*ckin, motherf***ing, etc., variations of "motherfucking"
    r'm[ou]*th[\!\@\#\$\%\^\&\*]*[a][\!\@\#\$\%\^\&\*]*f[\!\@\#\$\%\^\&\*]*[u][\!\@\#\$\%\^\&\*]*[c][\!\@\#\$\%\^\&\*]*[k][\!\@\#\$\%\^\&\*]*[i][\!\@\#\$\%\^\&\*]*[n][\!\@\#\$\%\^\&\*]*': 'motherfucking',
}

In [9]:
hindi_swear_map = {

    # —— “madarchod” ——
    # Roman + common leet/censor + Devanagari
    r'\b(m[a@]d[ae]?[r]?[\W_]*ch[o0]d[a@]?|म[ाअ]*दर[चच]ोद|मादरचोद)\b': 'madarchod',

    # —— “bhenchod / behenchod” ——
    r'\b(b[h]*e+h[e]*n[\W_]*ch[o0]d[a@]?|भेंचोद|बह[ेे]न[चच]ोद|बहनचोद)\b': 'bhenchod',

    # —— “chutiya / ch**iya” ——
    r'\b(c[h]*u+t+i+y*a+|ch[u*]+t+iy?[ae]+|चूतिया|चुतिया|छूतिया)\b': 'chutiya',

    # —— “chu***” shortened form ——
    r'\b(ch[\W_]*u[\W_]*t[\W_]*\*{2,})\b': 'chut***',

    # —— “gandu / gaandu / gaand” ——
    r'\b(g[a@]a*n+d[u]?[aei]?|गां*डू?|गाण्डू?)\b': 'gandu',

    # —— “gaand” stand‑alone  ——
    r'\b(ga+a+nd+|गांड|गांड|गाण्ड)\b': 'gaand',

    # —— “lauda / laudae / laude” —— 
    r'\b(l[a@]u+[d]+[aei]*|लौड़ा|लंड|लुंड)\b': 'lauda',

    # —— “lund” ——
    r'\b(l[u]n+d|लुंड|लंड)\b': 'lund',

    # —— “randi / r@ndi” ——
    r'\b(r[a@]n+d[iy]+|रंडी|रंड़ी)\b': 'randi',

    # —— “harami / haramzada / haramzadi” ——
    r'\b(h[a@]r+a+m[iy]*|हरामी)\b': 'harami',
    r'\b(h[a@]r+a+m[z$]?a+d[a@]|हरामज़ादा|हरामजादा)\b': 'haramzada',
    r'\b(h[a@]r+a+m[z$]?a+d[iy]|हरामज़ादी|हरामजादी)\b': 'haramzadi',

    # —— “kamine / kamina / kaminey” ——
    r'\b(k[a@]m+i+n[e]?[yie]*|कमीना|कमीने|कमीनी)\b': 'kamine',

    # —— “kutta / kutte / kuttiya” ——
    r'\b(k[u]t+t[a@e]*|कुत्ता|कुत्ते|कुते)\b': 'kutta',
    r'\b(k[u]t+t[i]y?a+|कुतिया|कुत्ती)\b': 'kutiya',

    # —— “saala / saale / sali” ——
    r'\b(s+a+a*l+a+|साला|साले)\b': 'saala',
    r'\b(s+a+l+i+|साली)\b': 'sali',

    # —— “bakchod / bakchodi” ——
    r'\b(b[a@]k+ch[o0]d[iy]*|बकचोद|बकचोदी)\b' : 'bakchod',

    # —— milder insults ——
    r'\b(g[a@]dh[a@]a+|गधा|गधे)\b': 'gadha',

    # —— catch‑all “f**k” in Roman Hindi sentences ——
    r'\b(f[\W_]*u[\W_]*c[\W_]*k+)\b': 'fuck',

    # —— emojis or censor blocks containing Devanāgarī letters and stars
    r'[\u0900-\u097F]\*{2,}[\u0900-\u097F]*': 'censored_hindi_word',
}


In [10]:
ascii_emoticons = {
    ':-)': 'smiley_face',
    ':)': 'smiley_face',
    ':-D': 'grinning_face',
    ':D': 'grinning_face',
    ':-(': 'sad_face',
    ':(': 'sad_face',
    ':-P': 'playful_face',
    ':P': 'playful_face',
    ':-p': 'playful_face',
    ':p': 'playful_face',
    ';-)': 'winking_face',
    ';)': 'winking_face',
    ':-O': 'surprised_face',
    ':O': 'surprised_face',
    ':-o': 'surprised_face',
    ':o': 'surprised_face',
    r':-/': 'skeptical_face',
    r':/': 'skeptical_face',
    r':-\|': 'neutral_face',
    r':\|': 'neutral_face',
    '<3': 'heart',
    '^_^': 'happy_face',
    '-_-': 'expressionless_face',
    'o_O': 'confused_face',
    'O_o': 'confused_face',
    'o_o': 'confused_face',
    '>_<': 'frustrated_face',
    'O.o': 'confused_face',
    'o.O': 'confused_face',
    '0_o': 'confused_face',
    'o_0': 'confused_face',
    'xD': 'laughing_face',
    'XD': 'laughing_face',
}


In [11]:
def replace_emojis(text):
    return emoji.demojize(text) if isinstance(text, str) else text

In [12]:
def normalize_text(text):
    if not isinstance(text, str):
        return text
    try:
        # Attempt to encode to 'latin1' and decode to 'utf-8' to fix encoding issues
        clean_text = text.encode('latin1').decode('utf-8')
    except (UnicodeEncodeError, UnicodeDecodeError):
        # Fallback: ignore encoding errors if they occur
        clean_text = text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
    return clean_text

In [13]:
def decode_html_entities(text):
    """Decode HTML entities in text, such as '&amp;' to '&'."""
    if not isinstance(text, str):
        return text
    # Use html.unescape to decode all HTML entities
    clean_text = html.unescape(text)
    return clean_text

In [14]:
def remove_html_tags(text):
    if not isinstance(text, str):
        return text
    clean_text = re.sub(r'</?[^>]+>', '', text)
    return clean_text

In [15]:
def replace_urls(text):
    if not isinstance(text, str):
        return text
    # Regex pattern to match URLs
    url_pattern = r'(http[s]?://\S+|www\.\S+)'
    clean_text = re.sub(url_pattern, ' SomeWebLink ', text)
    return clean_text

In [16]:
def replace_emails(text):
    if not isinstance(text, str):
        return text
    # Pattern to match email addresses
    email_pattern = r'[\w\.-]+@[\w\.-]+\.\w+'
    clean_text = re.sub(email_pattern, ' EMAILADDRESS ', text)
    return clean_text

In [17]:
def replace_censored_swear_words(text):
    if not isinstance(text, str):
        return text
    clean_text = text
    for pattern, replacement in censored_swear_words.items():
        # Do not add word boundaries if pattern already contains them
        if pattern.startswith(r'\b') and pattern.endswith(r'\b'):
            clean_pattern = pattern
        else:
            clean_pattern = r'\b' + pattern + r'\b'
        clean_text = re.sub(clean_pattern, replacement, clean_text, flags=re.IGNORECASE)
    return clean_text

In [18]:
def replace_hindi_swears(text: str) -> str:
    if not isinstance(text, str):
        return text
    out = text
    for pat, repl in hindi_swear_map.items():
        out = regex.sub(pat, repl, out, flags=regex.IGNORECASE)
    return out

In [19]:
def replace_mentions(text):
    """Replace mentions with 'SomeTaggedAccount', including when attached to other words."""
    if not isinstance(text, str):
        return text
    # Pattern to match @ followed by word characters
    # Replace mentions with spaces around to prevent word merging
    mention_pattern = r'@([A-Za-z0-9_]+)'
    clean_text = re.sub(mention_pattern, ' SomeTaggedAccount ', text)
    return clean_text

In [20]:
def replace_at_symbol(text):
    if not isinstance(text, str):
        return text
    # Replace '@' when used as 'at'
    # Conditions:
    # - '@' followed by a space
    # - '@' at the end of a string
    # - '@' followed by a number or time format
    # - '@' followed by a capitalized word (assuming location or time)
    clean_text = re.sub(r'@\s', 'at ', text)
    clean_text = re.sub(r'@$', 'at', clean_text)
    clean_text = re.sub(r'@(?=\d)', 'at ', clean_text)
    clean_text = re.sub(r'@ (?=[A-Z])', 'at ', clean_text)
    return clean_text

In [21]:
def remove_hashtags(text):
    if not isinstance(text, str):
        return text

    # Remove '#' symbol in various contexts
    clean_text = re.sub(r'(^|\s)#(\w*[^\s\w]?)', r'\1\2', text)
    clean_text = re.sub(r'(\w+)#([^\s\w]?)', r'\1\2', clean_text)
    clean_text = re.sub(r'([^\s\w])#', r'\1', clean_text)

    # Remove any remaining standalone '#' symbols
    clean_text = re.sub(r'(?<=\s)#(?=\s|$)|(?<=^)#(?=\s|$)', '', clean_text)

    return clean_text

In [22]:
def expand_contractions(text):
    if not isinstance(text, str):
        return text
    expanded_words = [contractions.fix(word) for word in text.split()]
    clean_text = ' '.join(expanded_words)
    return clean_text

In [23]:
def clean_unicode_punct(text: str) -> str:
    """
    Remove punctuation *except* ? ! . , and keep both Latin & Devanagari letters.
    Any other symbol is dropped.
    """
    if not isinstance(text, str):
        return text
    # keep letters, numbers, spaces and selected punctuation
    pattern = rf'[^\s{LATIN_BLOCK}{DEVANAGARI_BLOCK}0-9\?\!\.,]'
    return regex.sub(pattern, ' ', text)

In [24]:
def transliterate_devanagari(text: str) -> str:
    if not DEVANAGARI_TO_ROMAN or not isinstance(text, str):
        return text
    return transliterate(text, sanscript.DEVANAGARI, sanscript.ITRANS)

In [25]:
def remove_emoji_colons(text):
    """Remove colons around emoji descriptions in the text."""
    if not isinstance(text, str):
        return text
    # Regex pattern to match emoji descriptions with colons
    pattern = r':([a-zA-Z0-9_+-]+):'
    # Replace matches with the emoji description without colons
    clean_text = re.sub(pattern, r'\1', text)
    return clean_text

In [26]:
def replace_ascii_emoticons(text):
    """Replace ASCII emoticons in text with their descriptions."""
    if not isinstance(text, str):
        return text

    clean_text = text
    for emoticon, description in ascii_emoticons.items():
        # Escape the emoticon pattern to handle special characters
        escaped_emoticon = re.escape(emoticon)
        # Build the regex pattern to match emoticons not part of words
        pattern = r'(?<!\w)' + escaped_emoticon + r'(?!\w)'
        # Replace the emoticon with the description, ignoring case
        clean_text = re.sub(pattern, f"{description}", clean_text, flags=re.IGNORECASE)
    return clean_text

In [27]:
def replace_trade_mark(text):
    """Replace 'trade_mark' with a single quote (')."""
    if not isinstance(text, str):
        return text
    # Replace 'trade_mark' with "'"
    clean_text = text.replace("trade_mark", "'")
    return clean_text

In [28]:
def remove_extra_whitespace(text):
    if not isinstance(text, str):
        return text
    clean_text = ' '.join(text.split())
    return clean_text

In [29]:
clean_fns = [replace_emojis, normalize_text, decode_html_entities, remove_html_tags,
             replace_urls, replace_emails, replace_censored_swear_words,
             replace_hindi_swears, replace_mentions, replace_at_symbol,
             remove_hashtags, expand_contractions, clean_unicode_punct,
             transliterate_devanagari, remove_emoji_colons,
             replace_ascii_emoticons, replace_trade_mark, remove_extra_whitespace]

In [30]:
def clean(txt:str):
    for fn in clean_fns: txt = fn(txt)
    return txt.strip()

In [31]:
tfidf, svm = joblib.load(os.path.join(BASE_DIR, "tfidf_svm.joblib"))

In [32]:
import json, re
rows = np.load(os.path.join(BASE_DIR, "vocab.npy"), allow_pickle=True)
token2idx  = {k:int(v) for k,v in rows.tolist()}
EMB_DIM, MAXLEN = 300, 60
class BiLSTM(torch.nn.Module):
    def __init__(self, vocab_size, hidden=128):
        super().__init__()
        self.hidden = hidden                       # ← store
        self.embed  = torch.nn.Embedding(vocab_size, EMB_DIM, padding_idx=0)
        self.lstm   = torch.nn.LSTM(EMB_DIM, hidden, bidirectional=True, batch_first=True)
        self.fc     = torch.nn.Linear(hidden * 2, 2)
    def forward(self, x):
        e, _ = self.lstm(self.embed(x))
        h = torch.cat([e[:, -1, :self.hidden],      # ← use self.hidden
                       e[:,  0,  self.hidden:]], 1)
        return self.fc(h)
bilstm = BiLSTM(len(token2idx))
bilstm.load_state_dict(torch.load(os.path.join(BASE_DIR, "bilstm_state.pt"), map_location="cpu"))
bilstm.eval()

C:\Users\Manan\AppData\Local\Temp\ipykernel_7648\3531386633.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bilstm.load_state_dict(torch.load(os.path.join(BASE_DIR, "bi

BiLSTM(
  (embed): Embedding(5986, 300, padding_idx=0)
  (lstm): LSTM(300, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [33]:
def bilstm_encode(texts):
    ids, unk = [], token2idx.get("<unk>",1)
    for t in texts:
        toks = t.split()[:MAXLEN]
        vec  = [token2idx.get(tok, unk) for tok in toks] + [0]*(MAXLEN-len(toks))
        ids.append(vec)
    return torch.tensor(ids)

In [34]:
mb_path = os.path.join(BASE_DIR, "bert_mbert")
mbert_tok = AutoTokenizer.from_pretrained(mb_path)
mbert_net = AutoModelForSequenceClassification.from_pretrained(mb_path).eval()

In [35]:
hing_tok = AutoTokenizer.from_pretrained(HING_DIR)
hing_net = AutoModelForSequenceClassification.from_pretrained(HING_DIR).eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
bilstm, mbert_net, hing_net = bilstm.to(device), mbert_net.to(device), hing_net.to(device)

In [36]:
print("device:", device)

device: cuda


In [37]:
def predict_all(texts):
    cleaned = [clean(t) for t in texts]

    # SVM
    svm_pred = svm.predict(tfidf.transform(cleaned))

    # BiLSTM
    with torch.no_grad():
        logits = bilstm(bilstm_encode(cleaned).to(device))
        bil_pred = logits.argmax(1).cpu().numpy()

    # mBERT
    with torch.no_grad():
        mb = mbert_tok(cleaned, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        mb_pred = mbert_net(**mb).logits.argmax(1).cpu().numpy()

    # HingBERT
    with torch.no_grad():
        hg = hing_tok(cleaned, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        hg_pred = hing_net(**hg).logits.argmax(1).cpu().numpy()

    return {"SVM":svm_pred, "BiLSTM":bil_pred, "mBERT":mb_pred, "HingBERT":hg_pred}

In [38]:
edges  = [0, .2, .4, .6, .8, 1.01]
labels = ["0-20%", "20-40%", "40-60%", "60-80%", "80-100%"]
def get_bucket(r):           # r ∈ [0,1]
    for i in range(len(edges)-1):
        if edges[i] <= r < edges[i+1]:
            return labels[i]

bucket_pairs = {m:{b:[] for b in labels} for m in ["SVM","BiLSTM","mBERT","HingBERT"]}

LABEL_MAP = {"non_sarcastic":0, "sarcastic":1}

In [39]:
paths = sorted(glob.glob(os.path.join(BATCH_DIR, "labeled_batch_*.csv")))
for p in tqdm(paths, desc="batches"):
    df = pd.read_csv(p).dropna(subset=["sarcasm_label","Transliterated_Comment"])
    lbls = df["sarcasm_label"].str.strip().str.lower().map(LABEL_MAP)
    mask = lbls.notna()
    texts = df.loc[mask,"Transliterated_Comment"].astype(str)
    y_true= lbls[mask].astype(int).values

    ratios = texts.map(hindi_ratio).tolist()
    buckets= [get_bucket(r) for r in ratios]

    preds = predict_all(texts.tolist())
    for m, y_pred in preds.items():
        for b, yt, yp in zip(buckets, y_true, y_pred):
            bucket_pairs[m][b].append((yt, yp))

batches: 100%|██████████| 200/200 [27:22<00:00,  8.21s/it]


In [40]:
def f1(pairs):
    if not pairs: return np.nan
    yt, yp = zip(*pairs)
    _,_,f,_ = precision_recall_fscore_support(yt, yp, average="binary")
    return f

data = {m:[f1(bucket_pairs[m][b]) for b in labels] for m in bucket_pairs}
bucket_df = pd.DataFrame(data, index=labels)
display(bucket_df.round(3))

,SVM,BiLSTM,mBERT,HingBERT
0-20%,0.635,0.317,0.534,0.878
20-40%,0.664,0.565,0.771,0.904
40-60%,0.710,0.597,0.858,0.905
60-80%,0.796,0.671,0.891,0.944
80-100%,0.795,0.662,0.885,0.934
